In [2]:
# ============================================================
# FinMark: End-to-end Cleaning + Feature Engineering + Export
# Local JupyterLab (Windows) safe, with /mnt/data fallback
# ============================================================

import pandas as pd
import numpy as np
import os
from pathlib import Path

# Optional: opt-in to future behavior (prevents silent downcasting issues)
pd.set_option("future.no_silent_downcasting", True)

# ----------------------------
# 0) Paths (EDIT if needed)
# ----------------------------
feedback_path_win     = r"C:\Users\lawre\Downloads\Customer_Feedback_Data.csv"
product_path_win      = r"C:\Users\lawre\Downloads\Product_Offering_Data.csv"
transaction_path_win  = r"C:\Users\lawre\Downloads\Transaction_Data.csv"

# Optional fallback (for hosted/sandbox environments)
feedback_path_fb      = "/mnt/data/Customer_Feedback_Data.csv"
product_path_fb       = "/mnt/data/Product_Offering_Data.csv"
transaction_path_fb   = "/mnt/data/Transaction_Data.csv"

def pick_path(primary, fallback):
    if os.path.exists(primary): return primary
    if os.path.exists(fallback): return fallback
    raise FileNotFoundError(f"File not found:\n- {primary}\n- {fallback}")

feedback_path    = pick_path(feedback_path_win, feedback_path_fb)
product_path     = pick_path(product_path_win, product_path_fb)
transaction_path = pick_path(transaction_path_win, transaction_path_fb)

# Output folder (Windows-safe)
out_dir = Path.home() / "Downloads" / "finmark_outputs"
out_dir.mkdir(parents=True, exist_ok=True)

# ----------------------------
# 1) Load
# ----------------------------
fb_raw   = pd.read_csv(feedback_path)
prod_raw = pd.read_csv(product_path)
tx_raw   = pd.read_csv(transaction_path)

print("Loaded shapes:")
print("Feedback:", fb_raw.shape)
print("Products:", prod_raw.shape)
print("Transactions:", tx_raw.shape)

# ----------------------------
# 2) Helpers
# ----------------------------
notes = []

def std_title(series: pd.Series) -> pd.Series:
    """
    Standardize categorical text:
    - Use pandas StringDtype (prevents downcasting warnings)
    - Strip, Title-case
    - Replace common null tokens with pd.NA
    """
    s = series.astype("string")
    s = s.str.strip().str.title()
    s = s.replace(["Nan", "None", "Na", "N/A", "Null", ""], pd.NA)
    return s

def safe_numeric(series: pd.Series) -> pd.Series:
    return pd.to_numeric(series, errors="coerce")

# ----------------------------
# 3) Clean + Standardize
# ----------------------------

# ---- Transactions ----
tx = tx_raw.copy()

# Deduplicate
if "Transaction_ID" in tx.columns:
    before = len(tx)
    tx = tx.drop_duplicates(subset=["Transaction_ID"])
    notes.append(f"Dropped {before-len(tx)} duplicate Transaction_ID rows.")
else:
    before = len(tx)
    tx = tx.drop_duplicates()
    notes.append(f"Dropped {before-len(tx)} fully-duplicate transaction rows (no Transaction_ID).")

# Standardize date
if "Transaction_Date" in tx.columns:
    tx["Transaction_Date"] = pd.to_datetime(tx["Transaction_Date"], errors="coerce")
    bad_dates = tx["Transaction_Date"].isna().sum()
    if bad_dates:
        notes.append(f"Transaction_Date: {bad_dates} invalid dates set to NaT (coerced).")
else:
    notes.append("WARNING: Transaction_Date column not found. Time-based features limited.")

# Standardize amount (required)
if "Transaction_Amount" in tx.columns:
    tx["Transaction_Amount"] = safe_numeric(tx["Transaction_Amount"])
    before = len(tx)
    tx = tx.dropna(subset=["Transaction_Amount"])
    notes.append(f"Dropped {before-len(tx)} rows with missing/invalid Transaction_Amount.")
else:
    raise KeyError("Transaction_Data must contain Transaction_Amount.")

# Standardize transaction type
if "Transaction_Type" in tx.columns:
    tx["Transaction_Type"] = std_title(tx["Transaction_Type"])

# Key checks
if "Customer_ID" not in tx.columns:
    raise KeyError("Transaction_Data must contain Customer_ID.")

# ---- Products ----
prod = prod_raw.copy()

if "Product_ID" in prod.columns:
    before = len(prod)
    prod = prod.drop_duplicates(subset=["Product_ID"])
    notes.append(f"Dropped {before-len(prod)} duplicate Product_ID rows.")
else:
    before = len(prod)
    prod = prod.drop_duplicates()
    notes.append(f"Dropped {before-len(prod)} fully-duplicate product rows (no Product_ID).")

for c in ["Product_Name", "Product_Type", "Risk_Level", "Target_Age_Group", "Target_Income_Group"]:
    if c in prod.columns:
        prod[c] = std_title(prod[c])

if "Target_Age_Group" in prod.columns:
    miss = prod["Target_Age_Group"].isna().sum()
    if miss:
        prod["Target_Age_Group"] = prod["Target_Age_Group"].fillna("All")
        notes.append(f"Filled {miss} missing Target_Age_Group with 'All'.")

# ---- Feedback ----
fb = fb_raw.copy()

if "Feedback_ID" in fb.columns:
    before = len(fb)
    fb = fb.drop_duplicates(subset=["Feedback_ID"])
    notes.append(f"Dropped {before-len(fb)} duplicate Feedback_ID rows.")
else:
    before = len(fb)
    fb = fb.drop_duplicates()
    notes.append(f"Dropped {before-len(fb)} fully-duplicate feedback rows (no Feedback_ID).")

if "Feedback_Date" in fb.columns:
    fb["Feedback_Date"] = pd.to_datetime(fb["Feedback_Date"], errors="coerce")
    bad = fb["Feedback_Date"].isna().sum()
    if bad:
        notes.append(f"Feedback_Date: {bad} invalid dates set to NaT (coerced).")

if "Sentiment" in fb.columns:
    fb["Sentiment"] = std_title(fb["Sentiment"])

if "Rating" in fb.columns:
    fb["Rating"] = safe_numeric(fb["Rating"])

# Customer_ID in feedback is optional (features will be skipped if missing)
feedback_has_customer = "Customer_ID" in fb.columns
if not feedback_has_customer:
    notes.append("WARNING: Feedback has no Customer_ID; feedback features will be skipped.")

# ----------------------------
# 4) Feature Engineering (FinMark)
# ----------------------------

# Row-level time features
if "Transaction_Date" in tx.columns:
    tx["Tx_Month"] = tx["Transaction_Date"].dt.month
    tx["Tx_Year"]  = tx["Transaction_Date"].dt.year
    tx["Tx_DayOfWeek"] = tx["Transaction_Date"].dt.dayofweek  # 0=Mon

# Customer-level transaction aggregates
customer_tx = (
    tx.groupby("Customer_ID", as_index=False)
      .agg(
          Total_Transactions=("Transaction_Amount", "size"),
          Total_Spend=("Transaction_Amount", "sum"),
          Avg_Transaction_Value=("Transaction_Amount", "mean"),
          Max_Transaction_Value=("Transaction_Amount", "max"),
          Min_Transaction_Value=("Transaction_Amount", "min"),
          Spend_Volatility=("Transaction_Amount", "std"),
      )
)

# Recency / lifetime / intensity
if "Transaction_Date" in tx.columns:
    first_last = (
        tx.groupby("Customer_ID", as_index=False)
          .agg(
              First_Transaction_Date=("Transaction_Date", "min"),
              Last_Transaction_Date=("Transaction_Date", "max"),
          )
    )
    customer_tx = customer_tx.merge(first_last, on="Customer_ID", how="left")

    reference_date = tx["Transaction_Date"].max()
    customer_tx["Recency_Days"] = (reference_date - customer_tx["Last_Transaction_Date"]).dt.days

    customer_tx["Customer_Lifetime_Days"] = (
        (customer_tx["Last_Transaction_Date"] - customer_tx["First_Transaction_Date"]).dt.days
        .clip(lower=1)
    )

    customer_tx["Spend_Per_Day"] = customer_tx["Total_Spend"] / customer_tx["Customer_Lifetime_Days"]
    customer_tx["Transactions_Per_Month"] = (
        customer_tx["Total_Transactions"] / (customer_tx["Customer_Lifetime_Days"] / 30.0)
    )
else:
    notes.append("No Transaction_Date: skipped recency/lifetime/time-intensity features.")

# Spend concentration ratio
customer_tx["Max_to_Avg_Ratio"] = (
    customer_tx["Max_Transaction_Value"] /
    customer_tx["Avg_Transaction_Value"].replace(0, np.nan)
)

# Transaction-type behavior shares
if "Transaction_Type" in tx.columns:
    type_counts = (
        tx.pivot_table(index="Customer_ID", columns="Transaction_Type",
                       values="Transaction_Amount", aggfunc="size", fill_value=0)
        .reset_index()
    )
    type_cols = [c for c in type_counts.columns if c != "Customer_ID"]
    type_counts["Type_Total"] = type_counts[type_cols].sum(axis=1).replace({0: np.nan})

    for c in type_cols:
        safe = c.replace(" ", "_")
        type_counts[f"Share_{safe}"] = type_counts[c] / type_counts["Type_Total"]

    type_shares = type_counts[["Customer_ID"] + [c for c in type_counts.columns if c.startswith("Share_")]]
    customer_tx = customer_tx.merge(type_shares, on="Customer_ID", how="left")
else:
    notes.append("Transaction_Type not found; skipped transaction-type share features.")

# Feedback-derived features
customer_features = customer_tx.copy()

if feedback_has_customer:
    if "Rating" in fb.columns:
        fb_agg = (
            fb.groupby("Customer_ID", as_index=False)
              .agg(
                  Avg_Rating=("Rating", "mean"),
                  Feedback_Count=("Rating", "size"),
              )
        )
    else:
        fb_agg = fb.groupby("Customer_ID", as_index=False).agg(Feedback_Count=("Customer_ID", "size"))

    # Sentiment shares
    if "Sentiment" in fb.columns:
        sent = (
            fb.pivot_table(index="Customer_ID", columns="Sentiment",
                           values="Customer_ID", aggfunc="size", fill_value=0)
            .reset_index()
        )
        sent_cols = [c for c in sent.columns if c != "Customer_ID"]
        sent["Sent_Total"] = sent[sent_cols].sum(axis=1).replace({0: np.nan})
        for c in sent_cols:
            safe = c.replace(" ", "_")
            sent[f"Share_{safe}_Sentiment"] = sent[c] / sent["Sent_Total"]

        sent_shares = sent[["Customer_ID"] + [c for c in sent.columns if c.endswith("_Sentiment")]]
        fb_agg = fb_agg.merge(sent_shares, on="Customer_ID", how="left")

    customer_features = customer_features.merge(fb_agg, on="Customer_ID", how="left")

    # Fill missing feedback values
    if "Feedback_Count" in customer_features.columns:
        customer_features["Feedback_Count"] = customer_features["Feedback_Count"].fillna(0)

    if "Avg_Rating" in customer_features.columns:
        customer_features["Avg_Rating"] = customer_features["Avg_Rating"].fillna(customer_features["Avg_Rating"].mean())

    customer_features["Is_Silent_Customer"] = (customer_features.get("Feedback_Count", 0) == 0).astype(int)
else:
    notes.append("Skipped feedback features (no Customer_ID in feedback file).")

# ----------------------------
# 5) Merge: transaction_enriched (tx + product attributes)
# ----------------------------
if "Product_ID" in tx.columns and "Product_ID" in prod.columns:
    transaction_enriched = tx.merge(prod, on="Product_ID", how="left")
else:
    transaction_enriched = tx.copy()
    notes.append("Product_ID missing in tx or prod; skipped tx-product merge (transaction_enriched=tx).")

# ----------------------------
# 6) Export
# ----------------------------
customer_features_path     = out_dir / "FinMark_Customer_Features.csv"
transaction_enriched_path  = out_dir / "FinMark_Transaction_Enriched.csv"
products_cleaned_path      = out_dir / "FinMark_Product_Cleaned.csv"
feedback_cleaned_path      = out_dir / "FinMark_Feedback_Cleaned.csv"
notes_path                 = out_dir / "FinMark_Preprocessing_Notes.txt"

customer_features.to_csv(customer_features_path, index=False)
transaction_enriched.to_csv(transaction_enriched_path, index=False)
prod.to_csv(products_cleaned_path, index=False)
fb.to_csv(feedback_cleaned_path, index=False)

with open(notes_path, "w", encoding="utf-8") as f:
    f.write("FinMark Preprocessing Notes\n")
    f.write("="*30 + "\n")
    for n in notes:
        f.write(f"- {n}\n")

print("\n✅ Saved outputs to:", out_dir)
print(" -", customer_features_path.name, "shape:", customer_features.shape)
print(" -", transaction_enriched_path.name, "shape:", transaction_enriched.shape)
print(" -", products_cleaned_path.name, "shape:", prod.shape)
print(" -", feedback_cleaned_path.name, "shape:", fb.shape)
print(" -", notes_path.name)

display(customer_features.head())


Loaded shapes:
Feedback: (5050, 4)
Products: (15, 6)
Transactions: (5050, 5)

✅ Saved outputs to: C:\Users\lawre\Downloads\finmark_outputs
 - FinMark_Customer_Features.csv shape: (993, 20)
 - FinMark_Transaction_Enriched.csv shape: (4900, 8)
 - FinMark_Product_Cleaned.csv shape: (10, 6)
 - FinMark_Feedback_Cleaned.csv shape: (4969, 4)
 - FinMark_Preprocessing_Notes.txt


,Customer_ID,Total_Transactions,Total_Spend,Avg_Transaction_Value,Max_Transaction_Value,Min_Transaction_Value,Spend_Volatility,First_Transaction_Date,Last_Transaction_Date,Recency_Days,Customer_Lifetime_Days,Spend_Per_Day,Transactions_Per_Month,Max_to_Avg_Ratio,Share_Bill_Payment,Share_Investment,Share_Loan_Payment,Share_Purchase,Feedback_Count,Is_Silent_Customer
0,1,6,16836.0,2806.0,4993.0,156.0,2062.310646,2023-01-02 04:00:00,2023-07-02 03:00:00,26,180,93.533333,1.000000,1.779401,0.5,0.166667,0.166667,0.166667,2,0
1,2,2,4907.0,2453.5,2850.0,2057.0,560.735677,2023-02-06 04:00:00,2023-05-21 23:00:00,67,104,47.182692,0.576923,1.161606,0.5,0.000000,0.500000,0.000000,3,0
2,3,1,1538.0,1538.0,1538.0,1538.0,NaN,2023-02-27 23:00:00,2023-02-27 23:00:00,150,1,1538.000000,30.000000,1.000000,1.0,0.000000,0.000000,0.000000,3,0
3,4,2,8295.0,4147.5,4736.0,3559.0,832.264681,2023-01-22 15:00:00,2023-06-30 01:00:00,28,158,52.500000,0.379747,1.141893,0.0,0.000000,0.500000,0.500000,6,0
4,5,5,14798.0,2959.6,4878.0,1508.0,1386.584040,2023-02-19 15:00:00,2023-07-27 00:00:00,1,157,94.254777,0.955414,1.648196,0.4,0.200000,0.000000,0.400000,9,0
